# Direct Distances

In this approach, I make the assumption that the distance between two shops is independent of the road network.

## Generate Random Warehouses

In [1]:
import copy
import os
import pickle
import random
import os
from geopy.geocoders import Nominatim
from haversine import Unit, haversine_vector
import pulp as pl

In [2]:
if not os.path.exists("warehouses.pkl"):
    print("Generating warehouses info...")

    warehouses = []
    for i in range(1, 101):
        name = f"Warehouse {i}"
        lat = random.uniform(
            10.668799, 10.878041
        )  # Latitude range of Ho Chi Minh City
        long = random.uniform(
            106.451840, 106.813580
        )  # Longitude range of Ho Chi Minh City
        description = None
        warehouses.append(
            {
                "index": i,
                "name": name,
                "lat": lat,
                "long": long,
                "description": "",
            }
        )

    geolocator = Nominatim(user_agent="bk-imp")

    for i in range(len(warehouses)):
        lat, long = warehouses[i]["lat"], warehouses[i]["long"]
        location = geolocator.reverse(f"{lat}, {long}")
        address = location.address
        warehouses[i]["description"] = address

    with open("warehouses.pkl", "wb") as f:
        # Shuffle warehouses to generate random order
        random.shuffle(warehouses)
        pickle.dump(warehouses, f)
else:
    with open("warehouses.pkl", "rb") as f:
        warehouses = pickle.load(f)

In [3]:
with open("stores.pkl", "rb") as f:
    stores = pickle.load(f)

## Down Sample

In [4]:
# warehouses = random.choices(warehouses, k=10)
# stores = random.choices(stores, k=50)

## Calculate Distance Matrix

In [5]:
warehouses_stores_dinstance_matrix = haversine_vector(
    [(store["lat"], store["long"]) for store in stores],
    [(warehouse["lat"], warehouse["long"]) for warehouse in warehouses],
    Unit.KILOMETERS,
    comb=True,
)

In [6]:
warehouses_stores_dinstance_matrix.shape

(100, 1000)

## Linear Programming

In [7]:
# Set up the problem
problem = pl.LpProblem("Warehouse_Optimization", pl.LpMinimize)

### Define the environment

In [8]:
M = 100  # Number of warehouses
N = 1000 # Number of stores
warehouses = range(M)
w_cost = random.sample(range(100, 201), M)  # Operational cost of warehouses
w_capacity = random.sample(range(13000, 15001), M)  # Operational cost of warehouses
stores = range(N)
demands = random.sample(range(10, 1101), N)  # Daily demand of stores
d_cost = copy.copy(
    warehouses_stores_dinstance_matrix
)  # Delivery cost between warehouses vs stores

In [9]:
assert sum(w_capacity) > sum(demands), "All warehouses' capacity is less than stores' demands"

### Define the decision variables

In [10]:
wr = pl.LpVariable.dicts(
    "warehouse_store_connection", (warehouses, stores), cat="Binary"
)

### Define the objective function

In [11]:
total_cost = pl.lpSum(w_cost[w] * wr[w][s] + d_cost[w][s] * wr[w][s] for w in warehouses for s in stores)
problem += total_cost

### Define the constraints

In [12]:
# Each store must be assigned to one warehouse
for i in stores:
    problem += pl.lpSum(wr[j][i] for j in warehouses) == 1

In [13]:
# Total demand of assigned stores for a warehouse must be smaller than its capacity
for w in warehouses:
    problem += pl.lpSum(demands[s] * wr[w][s] for s in stores) <= w_capacity[w]

## Solve the problem!!!

### COIN-OR Branch and Cut solver (CBC)
https://coin-or.github.io/Cbc/intro.html#:~:text=The%20COIN%2DOR%20Branch%20and,executable%20version%20is%20also%20available.

In [14]:
problem.solve(pl.PULP_CBC_CMD(timeLimit=600, threads=6))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/terrabot/bk-imp/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/680e76e9074846879418bdac1795ea51-pulp.mps sec 600 threads 6 timeMode elapsed branch printingOptions all solution /tmp/680e76e9074846879418bdac1795ea51-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1105 COLUMNS
At line 501106 RHS
At line 502207 BOUNDS
At line 602208 ENDATA
Problem MODEL has 1100 rows, 100000 columns and 200000 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 600
threads was changed from 0 to 6
Option for timeMode changed from cpu to elapsed
Continuous objective value is 118613 - 0.22 seconds
Cgl0005I 1000 SOS with 100000 members
Cgl0004I processed model has 1100 rows, 100000 columns (100000 integer (100000 of which binary)) and 200000 elements
Cbc0038I Initial state - 72 integers unsatisfied sum - 15.2508
Cbc0038I Pass   1:

1

In [15]:
# Print the optimal solution
print("Optimal Solution:")
for w in warehouses:
    assigned_stores = []
    for s in stores:
        if pl.value(wr[w][s]) == 1:
            assigned_stores.append(s)
    print(f"Warehouse {w} serves these stores:", assigned_stores)

# Print the total cost
print("Total Cost: ", pl.value(problem.objective))

Optimal Solution:
Warehouse 0 serves these stores: [168, 191, 226, 321, 382, 413, 486, 490, 494, 503, 514, 518, 542, 591, 750, 786, 787, 789, 807, 828, 879, 897, 922, 989]
Warehouse 1 serves these stores: []
Warehouse 2 serves these stores: []
Warehouse 3 serves these stores: []
Warehouse 4 serves these stores: [1, 17, 62, 75, 84, 112, 187, 193, 196, 215, 251, 288, 303, 370, 391, 394, 545, 553, 560, 604, 607, 609, 628, 679, 685, 690, 726, 748, 820, 840, 904, 918, 954, 962, 965, 969]
Warehouse 5 serves these stores: [86, 89, 157, 181, 270, 277, 332, 509, 521, 641, 803, 812, 823, 839, 883, 941]
Warehouse 6 serves these stores: []
Warehouse 7 serves these stores: []
Warehouse 8 serves these stores: [47, 90, 214, 262, 281, 300, 347, 510, 662, 689, 824, 830, 861, 910]
Warehouse 9 serves these stores: []
Warehouse 10 serves these stores: [69, 126, 166, 216, 305, 333, 423, 476, 479, 484, 559, 618, 681, 742, 788, 927]
Warehouse 11 serves these stores: [78, 388, 502, 512, 532, 771, 817, 851, 97

### CPLEX Solver
Get it free from here https://www.ibm.com/academic/topic/data-science

or https://storage.googleapis.com/thaitang-sharing/cplex_studio2211.linux_x86_64.bin

In [16]:
os.environ['CPLEX_HOME'] = '/opt/ibm/ILOG/CPLEX_Studio2211/cplex'
os.environ['CPO_HOME'] = '/opt/ibm/ILOG/CPLEX_Studio2211/cpoptimizer'
os.environ['PATH'] += ':' + os.environ['CPLEX_HOME'] + '/bin/x86-64_linux:' + os.environ['CPO_HOME'] + '/bin/x86-64_linux'
os.environ['LD_LIBRARY_PATH'] += ':' + os.environ['CPLEX_HOME'] + '/bin/x86-64_linux:' + os.environ['CPO_HOME'] + '/bin/x86-64_linux'
os.environ['PYTHONPATH'] = '/opt/ibm/ILOG/CPLEX_Studio2211/cplex/python/3.10/x86-64_linux'

In [17]:
problem.solve(pl.CPLEX_CMD(timeLimit=600, threads=6))


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Problem '/tmp/da2b6af3f05246ac8e308a246b13268f-pulp.lp' read.
Read time = 0.18 sec. (5.44 ticks)
CPLEX> New value for time limit in seconds: 600
CPLEX> New value for default parallel thread count: 6
CPLEX> Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Threads                                 6
CPXPARAM_TimeLimit                               600
Tried aggregator 1 time.
Reduced MIP has 1100 rows, 100000 columns, and 200000 nonzeros.
Reduced MIP has 100000 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.30 sec. (176.49 ticks)
Found incumbent of value 165930.196287 after 0.76 sec. (420.76 ticks)
Tr

1

In [18]:
# Print the optimal solution
print("Optimal Solution:")
for w in warehouses:
    assigned_stores = []
    for s in stores:
        if pl.value(wr[w][s]) == 1:
            assigned_stores.append(s)
    print(f"Warehouse {w} serves these stores:", assigned_stores)

# Print the total cost
print("Total Cost: ", pl.value(problem.objective))

Optimal Solution:
Warehouse 0 serves these stores: [168, 191, 213, 226, 321, 382, 413, 486, 490, 494, 514, 542, 591, 740, 786, 787, 789, 828, 846, 879, 897, 922, 974, 989]
Warehouse 1 serves these stores: []
Warehouse 2 serves these stores: []
Warehouse 3 serves these stores: []
Warehouse 4 serves these stores: [1, 17, 62, 84, 112, 187, 193, 196, 215, 251, 288, 303, 368, 370, 391, 394, 485, 545, 553, 560, 604, 607, 609, 628, 679, 685, 726, 748, 820, 840, 904, 918, 954, 962, 965, 969]
Warehouse 5 serves these stores: [86, 89, 157, 270, 277, 332, 509, 521, 641, 803, 812, 823, 832, 883, 899, 941, 959]
Warehouse 6 serves these stores: []
Warehouse 7 serves these stores: []
Warehouse 8 serves these stores: [47, 90, 214, 262, 281, 300, 510, 662, 689, 830, 843, 861, 910]
Warehouse 9 serves these stores: []
Warehouse 10 serves these stores: [69, 166, 216, 227, 305, 318, 333, 414, 423, 484, 559, 655, 681, 742, 788, 927]
Warehouse 11 serves these stores: [27, 78, 388, 512, 771, 851, 878, 975, 99